# A Basic Approach of a beginner getting <span style="color:red">97,5% </span>.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

load the Data 

In [ ]:
df0 = pd.read_csv('../input/credit-card-customers/BankChurners.csv')
df0

In [ ]:
df= df0.copy()

# Start Exploratory analysis of the Data : 

in this Step we need to collect some information from our Data. Graph Visualisation(Heatmap, box, bar,Line...) will help us a lot in this Step. 
Take notes of :

   * Data Type  (Numerical/Categorical) 
   
and check : 

   * Missing Data 
   * Distribution 
   * Relation between Features (Correlations)             
   * Outleirs 
   * Balance / Imbalance Data  
   


In [ ]:
df.isnull().sum() 

In [ ]:
 # delet some unusefull Columns 
del df['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2']
del df['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1']
del df['CLIENTNUM']

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
count=pd.value_counts(df['Attrition_Flag']).tolist()
plt.figure(figsize=(5,5))
plt.title("Percentage of Attrited Customer and Existing Customer")
plt.pie(x=count,labels=["Attrited Customer","Existing Customers"],autopct='%.2f%%')

In [ ]:
import seaborn as sns
plt.figure(figsize=(28,11))
plt.title("Distribution of Age with respect to Churned or not")
sns.countplot(data=df,x=df["Customer_Age"],hue="Attrition_Flag")

In [ ]:

cats = ['Gender','Education_Level','Marital_Status', 'Income_Category', 'Card_Category','Attrition_Flag']

fig, axis = plt.subplots(len(cats) // 3,3, figsize=(20,12))
fig.suptitle('The distibution of the Churners')

index = 0
for i in range(len(cats) // 3):
      for j in range(3):
            
            ax = sns.countplot(ax=axis[i][j] ,x=df[cats[index]], hue=df['Attrition_Flag'],palette="Set1")
            ax.legend(title='Customer',loc='upper right',labels=['Existing', 'attired'])
            for p in ax.patches:
                height = p.get_height()
                ax.text(p.get_x()+p.get_width()/2.,
                        height + 3,
                        '{:1.2f}%'.format(height/len(df)*100),
                        ha="center") 

In [ ]:
#Création de sous-ensembles positifs et négatifs

numr = ['Months_Inactive_12_mon','Contacts_Count_12_mon','Credit_Limit',
                  'Total_Amt_Chng_Q4_Q1','Total_Trans_Ct','Total_Ct_Chng_Q4_Q1']

existing_df = df[df['Attrition_Flag'] == 'Existing Customer']
attired_df = df[df['Attrition_Flag'] == 'Attrited Customer']


for col in numr:
    plt.figure()
    sns.distplot(existing_df[col], label='existing')
    sns.distplot(attired_df[col], label=' attired')
    plt.legend()

In [ ]:
import seaborn as sns


fig, axis = plt.subplots(len(cats) // 3,3, figsize=(20,12))
fig.suptitle('The distibution of the numr')
sns.set_style("darkgrid", {"axes.facecolor": ".9"})


index = 0
for i in range(len(cats) // 3):
      for j in range(3):
            
            ax = sns.boxplot(ax=axis[i][j] ,x=df[numr[index]])
            index += 1
            #sns.set_theme(style="dark")

In [ ]:
#outlier cleanup
from scipy import stats
import numpy as np
columns = ["Customer_Age", 'Dependent_count', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']
print(df.shape)
for column in columns : 
    z = np.abs(stats.zscore(df[column]))
    df=df[(z < 3)]
print(df.shape)

# Preprocessing & Feature engineering


In [ ]:
df_cat=df.select_dtypes(include=[object])
df_numr=df.select_dtypes(exclude=[object])

In [ ]:
# one hot encoding of object category data
obj_data=pd.get_dummies(df_cat.drop(columns=['Attrition_Flag']),drop_first=True)
obj_data


In [ ]:
# joining our numr dataframe and object dataframe
df=pd.concat([obj_data,df_numr],axis=1)
df.shape

In [ ]:
# splitting dependent and independent feature 

Y=pd.get_dummies(df_cat['Attrition_Flag'],drop_first=True)
X=df

print(X.shape)
print(Y.shape)

In [ ]:
# dividing dataset into training and testing set 
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
from imblearn.combine import SMOTETomek
from collections import Counter 


smk=SMOTETomek(random_state=42)
xres,yres=smk.fit_sample(X_train,y_train.values.ravel())

print(yres.shape, xres.shape)
print('Old Shape {}'.format(Counter(Y)))
print('Resampled Shape {}'.format(Counter(yres)))

# BUILDING MODELS & EVALUATION


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import PolynomialFeatures, StandardScaler 

In [ ]:
preprocessor =  make_pipeline(SelectKBest(f_classif, k=10) ,PolynomialFeatures(2))


AdaBoost = make_pipeline(preprocessor,AdaBoostClassifier(random_state=0))
SVM= make_pipeline(preprocessor, StandardScaler(), SVC(random_state=0))
GBoost = make_pipeline(preprocessor, StandardScaler(), GradientBoostingClassifier())
RandomForest = make_pipeline( preprocessor, RandomForestClassifier())
XGB = make_pipeline( preprocessor, XGBClassifier())
Extree = make_pipeline( preprocessor, ExtraTreesClassifier())

dict_of_models = {'AdaBoost':AdaBoost,
                  'SVM':SVM,
                  'GBoost':GBoost,
                  'RandomForest':RandomForest,
                  'XGB':XGB,
                  'Extree':Extree}

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV

In [ ]:
def evaluation(model_1):
    
    #np.seterr(divide='ignore', invalid='ignore')
    model_1.fit(xres, yres)
    ypred = model_1.predict(X_test)
    cm = confusion_matrix(y_test, ypred)
    #sns.heatmap(cm,annot=True, annot_kws={"size": 16})
    N, train_score, val_score = learning_curve(model_1, xres, yres,
                                              cv=4, scoring='f1',
                                               train_sizes=np.linspace(0.1, 1, 10))
    
    
    plt.figure(figsize=(12, 8))
    plt.plot(N, train_score.mean(axis=1), label='train score')
    plt.plot(N, val_score.mean(axis=1), label='validation score')
    plt.legend()
    
    
    print(confusion_matrix(y_test, ypred))
    print(classification_report(y_test, ypred))

In [ ]:

for name, model in dict_of_models.items():
    print(name)
    evaluation(model)


# Choose the Best MODEL and Tune Hyperparamater

In [ ]:

params = {"max_depth" : [6,7,8],
          "min_child_weight" : [1,2,3],
          'gamma ':[0],
          'subsample':[0.6,0.7,0.8],
          #'colsample_bytree':
         }
    
         

clf = XGBClassifier()
grid= GridSearchCV (clf, params,scoring="recall", cv = 3)

grid_result = grid.fit(xres, yres)




#Summary 
#print("classifcation report: ",classification_report(y_test, y_pred))
print("Best: %f using %s " % (grid_result.best_score_, grid_result.best_params_) )
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]

for mean, stdev, param in zip (means, stds, params):
    print("%f (%f) with : %r" % (mean, stdev, param))

In [ ]:
from sklearn.metrics import*
from sklearn.model_selection import* 
from sklearn.preprocessing import*


xgb = XGBClassifier(gamma = 0, max_depth=7, min_child_weight= 1, subsample= 0.8) 
xgb.fit(xres, yres)
y_pred = xgb.predict(X_test)

acc = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test.values, y_pred)

print("Acc  : ", acc)

In [ ]:
pd.DataFrame(xgb.feature_importances_, index=X_train.columns).plot.bar(figsize=(12, 8))